In [7]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pyodbc
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import numpy as np

In [8]:
def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor

db_connection = database_connection()

In [9]:
user_id=100
#items tracking
sql = "EXEC [dbo].[usp_GetCurrentTracking] {}".format(user_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
#df = pd.Series(list_items)
labels = ['item id','item name']
df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list
print(df)    

   item id             item name
0      101     Adrenaline GTS 17
1      102  Radian R100 Car Seat
2      103               Samsung


In [21]:
#set up user and item we are looking up
item_id = 102


In [22]:
#price history of items
sql = "EXEC [dbo].[usp_GetUserItemPriceHistory] {},{}".format(user_id,item_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result

#putting price history into a data frame
labels = ['item id','item name','Brand','Store ID', 'Store Name','Item Price','Price Date']
price_history_df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list


#pivot data and replace all None values with 0
pvt_price_history =  price_history_df.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)

print(pvt_price_history)
#unpvit the price 
upvt_price_history = pvt_price_history.unstack().reset_index(name='Item Price')

print(upvt_price_history)

#replace 0 with None so we can front fill later
upvt_price_history['Item Price'].replace(
    to_replace=[0],
    value=None,
    inplace=True
)

print(upvt_price_history)

#front fill all the data
upvt_price_history.groupby('Price Date')['Item Price'].fillna(method='bfill') 

print(upvt_price_history)
# repivot for graphing
pvt_price_history2 =  upvt_price_history.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)

print(pvt_price_history2)


Store Name  Amazon  Target Toys r Us Walmart
Price Date                                  
2018-03-08       0  239.99    179.99       0
2018-03-09       0  239.99    179.99       0
2018-03-12  179.99  234.99    179.99  179.99
2018-03-13  209.99  234.99         0       0
2018-03-14  209.99  234.99         0       0
2018-03-15  209.99  234.99         0  234.99
2018-03-17  209.99  234.99    234.99  234.99
2018-03-18  209.99  234.99         0  234.99
2018-03-19  209.99  234.99         0  234.99
   Store Name  Price Date Item Price
0      Amazon  2018-03-08          0
1      Amazon  2018-03-09          0
2      Amazon  2018-03-12     179.99
3      Amazon  2018-03-13     209.99
4      Amazon  2018-03-14     209.99
5      Amazon  2018-03-15     209.99
6      Amazon  2018-03-17     209.99
7      Amazon  2018-03-18     209.99
8      Amazon  2018-03-19     209.99
9      Target  2018-03-08     239.99
10     Target  2018-03-09     239.99
11     Target  2018-03-12     234.99
12     Target  2018-03-1

In [18]:
amazon = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Amazon']
walmart = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Walmart']
toysrus = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Toys r Us']
target = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Target']
item_name = price_history_df["item name"].unique()[0]

amazon_trace = go.Scatter(
    x = list(amazon['Price Date']),#dates
    y = list(amazon['Item Price']),
    name = 'Amazon',
    line = dict(
        color = ('rgb(49, 86, 219)'),
        width = 4)
)


walmart_trace = go.Scatter(
    x = list(walmart['Price Date']),#dates
    y = list(walmart['Item Price']),
    name = 'Walmart',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

toys_trace = go.Scatter(
    x = list(toysrus['Price Date']),#dates
    y = list(toysrus['Item Price']),
    name = 'Toys r Us',
    line = dict(
        color = ('rgb(49, 177, 32)'),
        width = 4)
)

target_trace = go.Scatter(
    x = list(target['Price Date']),#dates
    y = list(target['Item Price']),
    name = 'Target',
    line = dict(
        color = ('rgb(255, 153, 0)'),
        width = 4)
)


data = [amazon_trace, walmart_trace, toys_trace, target_trace]

layout = go.Layout(
    title='Price History for {}'.format(item_name),
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Item Price',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-line')